### tf-idf, 추출 순서를 맞추기

In [1]:
import requests
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.text import Tokenizer
import time
import numpy as np
import operator
from konlpy.tag import Komoran
from urllib.parse import quote_plus
import lxml.html
import pickle
import re
import keras
import pandas as pd

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [ ]:
"""
기존의 키워드가 너무 많다고 판단하였다. 따라서 기존의 tf-idf를 통해서 나온 키워드를 사용하지만 명사의 출현 순서를 고려하여 키워드 벡터를 다시 만든다.
일단 기존의 키워드 개수 20개를 그대로 가지고 간다는 것으로 생각하자
"""

In [2]:
## pickle load 및 확인
with open('Keyword0523.txt', 'rb') as f:
    keyword = pickle.load(f)
    
with open('para0523.txt', 'rb') as f:
    para = pickle.load(f)
    para = list(para)

print(len(keyword))
print(len(para))
print(keyword[0])
print(para[0])

17090
17090
['시공', '대회', '가치관', '건물', '건축', '투영', '경진', '중학', '경지', '건설', '일용직', '놀라움', '충남', '시절', '은사', '무임승차', '현장', '벽돌', '인생', '하자']
‘무임승차하지 않는 삶’ 중학 시절, 은사님께서 해주신 말씀입니다. 직접 노력해서 결과를 얻어야 한다는 기본적 원칙은 곧 제 인생의 가치관이 되었습니다. 인생의 가치관이 생기자 새로운 목표가 생겼습니다. 그 계기는 고등학생 시절 학창시절 우연히 하게 된 일용직 근로자 경험에서 시작되었습니다. 작은 벽돌이 쌓여서 큰 건물이 완성되는 모습은 저에겐 큰 놀라움이었습니다. 당시 어린 눈으로 봤던 건설현장은 저의 가치관이 그대로 투영된 곳으로 다가왔고 곧, 건설현장은 저에게 목표가 되었습니다. 저는 건축엔지니어로서의 기본을 다지기 위해 아래와 같은 노력을 했습니다. 다양한 분야의 지식을 습득했습니다. 건축은 구조, 시공, 안전 등 다양한 분야로 이루어져 있습니다. 그 때문에 한 가지 분야만 능통해서는 온전한 건물을 지을 수 없습니다. 그래서 저는 설계, 안전, 환경, 시공 등 다양한 전공수업을 이수하였습니다. 다양한 변수가 있는 현장에서 습득한 지식을 토대로 공기단축에 이바지하도록 하겠습니다. 다양한 경험을 위해 경진대회에 참여했습니다. 학부 시절 시공경진 대회에 참여했습니다. 대회를 준비하는 과정에서의 공부는 시공과 하자에 대한 다양한 지식을 얻을 수 있었습니다. 또한, 21개 팀이 참가한 경진대회에서 100명이 넘는 인원과 충남지역 대기업 실무자분들 앞에서 발표한 경험은 저를 더욱 단단하게 만들어 주었고, '대상'이라는 성과도 얻을 수 있었습니다.


In [3]:
tagger = Komoran()

def get_noun(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) >= 2]  # 명사만 추출

words = []
for i in range(0, len(para)):
    a = get_noun(para[i])
    words.append(a)

In [4]:
new_words = []
for i in range(0, len(words)):
    a = []
    for j in range(0, len(words[i])):
        if words[i][j] in keyword[i]:
            if words[i][j] not in a:
                a.append(words[i][j])
    new_words.append(a)
print('개수는 :', len(new_words))

개수는 : 17090


In [5]:
print(new_words[0:5])

[['무임승차', '중학', '시절', '은사', '인생', '가치관', '벽돌', '건물', '놀라움', '건설', '현장', '투영', '건축', '시공', '경진', '대회', '경지', '하자', '충남'], ['안전', '현직', '교수', '적용', '교육법', '접근법', '공사', '성패', '무재', '교양', '과목', '창의', '발명', '재산', '비판', '브레인스토밍', '개인', '교회', '인화', '단결'], ['공모전', '리더', '부지', '선정', '공법', '준비', '범위', '제출', '과업', '부분', '점검', '와중', '작품', '순위', '공사', '소통', '난관', '다수', '협력', '건설'], ['건축', '전문', '품질', '확보', '안전', '공사', '현장', '관리자', '기사', '교육', '유치원', '난이도', '무사', '무재', '지지부진', '한동안', '건설업', '침체', '하자', '최소'], ['교회', '청년', '상황', '침체', '출석', '풋살', '볼링', '친선경기', '운동회', '올림픽', '행사', '애착', '주인', '의식', '평일', '이전', '민원', '안전사고', '임무', '완수']]


In [6]:
# 실직적으로 20개 되지 않는 index들
index = []
for i in range(0, len(new_words)):
    if len(new_words[i]) != 20:
        index.append(i)
print("20개가 되지 않는 개수 :", len(index))

20개가 되지 않는 개수 : 4730


In [7]:
"""
20개씩 키워드를 정렬하였지만 20개가 아닌 것들이 무려 4730개가 되었다.
따라서 어떤 인덱스에서 최소 값과 최대 값을 알아보자
"""

'\n20개씩 키워드를 정렬하였지만 20개가 아닌 것들이 무려 4730개가 되었다....\n따라서 어떤 인덱스에서 최소 값과 최대 값을 알아보자\n'

In [8]:
# 각 index의 길이 알아보기
index_length = []
for i in range(0, len(index)):
    index_length.append(len(new_words[i]))

print("최소값 :", min(index_length))
print("최대값 :", max(index_length))

최소값 : 9
최대값 : 20


In [3]:
"""
길이의 최소값은 9이고 최대값은 20이다.
"""

'\n길이의 최소값은 9이고 최대값은 20이다...\n'

In [10]:
# 20개 나오지 않은 데이터 삭제
index.reverse()
for i in index:
    del new_words[i]
    del para[i]
print(len(new_words))
print(len(para))

12360
12360


In [13]:
# Words Word2Vec
embedding_model = Word2Vec(new_words,
                           size = 100,  # 100차원의 벡터로 바꾼다
                           window = 2,  # 주변 단어 앞 뒤로 두개까지
                           min_count = 1,  # 출현 빈도의 최소값, 나머진 삭제
                           workers = 4,  # CPU는 쿼드코어를 사용
                           iter = 100, # 100번 학습
                           sg = 1 # 분석방법론은 CBOW와 Skip-Gram 중 후자 선택
                          )

print("embedding model 학습 완료!")

# 단어를 벡터 값으로 바꾸기
words_vec = []
for i in range(0, len(new_words)):
    words_vec_before = []
    
    for j in range(0, len(new_words[i])):
        a = embedding_model[new_words[i][j]]
        for k in range(0, len(a)):
            words_vec_before.append(a[k])
                
    words_vec.append(words_vec_before)
    
print("단어를 벡터로 하는 list 완료!")
    
# padding
words_vec_length_count = []
for i in range(0, len(words_vec)):
    words_vec_length_count.append(len(words_vec[i]))

vec_max_length = max(words_vec_length_count)

for i in range(0, len(words_vec)):
    if len(words_vec[i]) != vec_max_length:
        for j in range(0, (vec_max_length-len(words_vec[i]))):
            words_vec[i].append(0)
            
print("padding 완료!")
print("만들어진 dense vector의 shape :", len(words_vec), "X", len(words_vec[0]))

embedding model 학습 완료!


/opt/venv/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


단어를 벡터로 하는 list 완료!
padding 완료!
만들어진 dense vector의 shape : 12360 X 2000


In [14]:
# pickle에 저장하기
with open('test0607.txt', 'wb') as f:
    pickle.dump(words_vec, f)
f.close

with open('para0607.txt', 'wb') as f:
    pickle.dump(para, f)
f.close

<function BufferedWriter.close>

In [11]:
with open('keywords0607.txt', 'wb') as f:
    pickle.dump(new_words, f)
f.close

<function BufferedWriter.close>